In [1]:
import numpy as np
import pandas as pd

In [2]:
def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame(prices.copy())
    df['upr'] = df.high.rolling(period).max().shift(periods=1)
    df['lwr'] = df.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df


def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    return df

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
periods = pd.Series({'1': 48, '2': 24, '3': 12, '4': 6})  # System look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly prices.
prices = weekly(exchange, tidm)

In [5]:
# Donchian channels.
dc = pd.Series(index=periods.index, dtype='object')
for sys, period in periods.items():
    dc.at[sys] = donchian(prices, period)

In [7]:
# Trade signals.
for sys, donchian in dc.items():
    if sys == 1:
        dc.loc[f'{sys}']['buy'] = np.where(dc.loc[f'{sys}'].close > dc.loc[f'{sys}'].upr, 1, 0)
    else:
        dc.loc[f'{sys}']['buy'] =  dc.loc[f'{sys}'].entry
    dc.loc[f'{sys}']['sell'] = np.where(dc.loc[f'{sys}'].close < dc.loc[f'{sys}'].mid, 1, 0)
    dc.loc[f'{sys}']['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
dc.loc['1']

AttributeError: 'DataFrame' object has no attribute 'entry'